In [8]:
# Author: Elisabetta Ghisu

"""
- This script take as input a kernel matrix
and returns the classification or regression performance
- The kernel matrix can be calculated using any of the graph kernels approaches
- The criteria used for prediction are SVM for classification and kernel Ridge regression for regression
- For predition we divide the data in training, validation and test. For each split, we first train on the train data, 
then evaluate the performance on the validation. We choose the optimal parameters for the validation set and finally
provide the corresponding performance on the test set. If more than one split is performed, the final results 
correspond to the average of the performances on the test sets. 

@references
    https://github.com/eghisu/GraphKernels/blob/master/GraphKernelsCollection/python_scripts/compute_perf_gk.py
"""

print(__doc__)

import sys
import os
import pathlib
sys.path.insert(0, "../")
from tabulate import tabulate

import random
import numpy as np
import matplotlib.pyplot as plt

from sklearn.kernel_ridge import KernelRidge # 0.17
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn import svm

from pygraph.kernels.marginalizedKernel import marginalizedkernel
from pygraph.utils.graphfiles import loadDataset

print('\n Loading dataset from file...')
dataset, y = loadDataset("../../../../datasets/acyclic/Acyclic/dataset_bps.ds")
y = np.array(y)
print(y)

# setup the parameters
model_type = 'regression' # Regression or classification problem
print('\n --- This is a %s problem ---' % model_type)

datasize = len(dataset)
trials = 100 # Trials for hyperparameters random search
splits = 10 # Number of splits of the data
alpha_grid = np.logspace(-10, 10, num = trials, base = 10) # corresponds to (2*C)^-1 in other linear models such as LogisticRegression
C_grid = np.logspace(-10, 10, num = trials, base = 10)
random.seed(20) # Set the seed for uniform parameter distribution

# set the output path
kernel_file_path = 'kernelmatrices_marginalized_acyclic/'
if not os.path.exists(kernel_file_path):
    os.makedirs(kernel_file_path)


"""
-  Here starts the main program
-  First we permute the data, then for each split we evaluate corresponding performances
-  In the end, the performances are averaged over the test sets
"""
val_means_pquit = []
val_stds_pquit = []
test_means_pquit = []
test_stds_pquit = []


for p_quit in np.linspace(0.1, 0.9, 9):
    print('\n --- calculating kernel matrix when termimation probability = %.1f ---' % p_quit)

    # save kernel matrices to files / read kernel matrices from files
    kernel_file = kernel_file_path + 'p_quit-' + str(p_quit)
    path = pathlib.Path(kernel_file)
    # get train set kernel matrix
    if path.is_file():
        print('\n Loading the kernel matrix from file...')
        Kmatrix = np.loadtxt(kernel_file)
        print(Kmatrix)
    else:
        print('\n Calculating kernel matrix, this could take a while...')
        Kmatrix = marginalizedkernel(dataset, p_quit, 20)
        print(Kmatrix)
        print('\n Saving kernel matrix to file...')
        np.savetxt(kernel_file, Kmatrix)

    # Initialize the performance of the best parameter trial on validation with the corresponding performance on test
    val_split = []
    test_split = []

    # For each split of the data
    for j in range(10, 10 + splits):
#         print('\n Starting split %d...' % j)

        # Set the random set for data permutation
        random_state = int(j)
        np.random.seed(random_state)
        idx_perm = np.random.permutation(datasize)
    #     print(idx_perm)

        # Permute the data
        y_perm = y[idx_perm] # targets permutation
    #     print(y_perm)
        Kmatrix_perm = Kmatrix[:, idx_perm] # inputs permutation
    #     print(Kmatrix_perm)
        Kmatrix_perm = Kmatrix_perm[idx_perm, :] # inputs permutation

        # Set the training, validation and test
        # Note: the percentage can be set up by the user
        num_train_val = int((datasize * 90) / 100)         # 90% (of entire dataset) for training and validation
        num_test = datasize - num_train_val              # 10% (of entire dataset) for test
        num_train = int((num_train_val * 90) / 100) # 90% (of train + val) for training
        num_val = num_train_val - num_train # 10% (of train + val) for validation

        # Split the kernel matrix
        Kmatrix_train = Kmatrix_perm[0:num_train, 0:num_train]
        Kmatrix_val = Kmatrix_perm[num_train:(num_train + num_val), 0:num_train]
        Kmatrix_test = Kmatrix_perm[(num_train + num_val):datasize, 0:num_train]

        # Split the targets
        y_train = y_perm[0:num_train]

        # Normalization step (for real valued targets only)
        if model_type == 'regression':
#             print('\n Normalizing output y...')
            y_train_mean = np.mean(y_train)
            y_train_std = np.std(y_train)
            y_train = (y_train - y_train_mean) / float(y_train_std)
    #         print(y)

        y_val = y_perm[num_train:(num_train + num_val)]
        y_test = y_perm[(num_train + num_val):datasize]

        # Record the performance for each parameter trial respectively on validation and test set
        perf_all_val = []
        perf_all_test = []

        # For each parameter trial
        for i in range(trials):
            # For regression use the Kernel Ridge method
            if model_type == 'regression':
    #             print('\n Starting experiment for trial %d and parameter alpha = %3f\n ' % (i, alpha_grid[i]))

                # Fit the kernel ridge model
                KR = KernelRidge(kernel = 'precomputed', alpha = alpha_grid[i])
#                 KR = svm.SVR(kernel = 'precomputed', C = C_grid[i])
                KR.fit(Kmatrix_train, y_train)

                # predict on the validation and test set
                y_pred = KR.predict(Kmatrix_val)
                y_pred_test = KR.predict(Kmatrix_test)
    #             print(y_pred)

                # adjust prediction: needed because the training targets have been normalizaed
                y_pred = y_pred * float(y_train_std) + y_train_mean
    #             print(y_pred)
                y_pred_test = y_pred_test * float(y_train_std) + y_train_mean
    #             print(y_pred_test)

                # root mean squared error on validation
                rmse = np.sqrt(mean_squared_error(y_val, y_pred))
                perf_all_val.append(rmse)

                # root mean squared error in test 
                rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))
                perf_all_test.append(rmse_test)

    #             print('The performance on the validation set is: %3f' % rmse)
    #             print('The performance on the test set is: %3f' % rmse_test)

        # --- FIND THE OPTIMAL PARAMETERS --- #
        # For regression: minimise the mean squared error
        if model_type == 'regression':

            # get optimal parameter on validation (argmin mean squared error)
            min_idx = np.argmin(perf_all_test)
            alpha_opt = alpha_grid[min_idx]

            # performance corresponding to optimal parameter on val
            perf_val_opt = perf_all_val[min_idx]

            # corresponding performance on test for the same parameter
            perf_test_opt = perf_all_test[min_idx]

#             print('The best performance is for trial %d with parameter alpha = %3f' % (min_idx, alpha_opt))
#             print('The best performance on the validation set is: %3f' % perf_val_opt)
#             print('The corresponding performance on test set is: %3f' % perf_test_opt)
        
        # append the best performance on validation
        # at the current split
        val_split.append(perf_val_opt)

        # append the correponding performance on the test set
        test_split.append(perf_test_opt)
        
    # average the results
    # mean of the validation performances over the splits
    val_mean = np.mean(np.asarray(val_split))
    # std deviation of validation over the splits
    val_std = np.std(np.asarray(val_split))

    # mean of the test performances over the splits
    test_mean = np.mean(np.asarray(test_split))
    # std deviation of the test oer the splits
    test_std = np.std(np.asarray(test_split))
    
    print('\n Mean performance on val set: %3f' % val_mean)
    print('With standard deviation: %3f' % val_std)
    print('\n Mean performance on test set: %3f' % test_mean)
    print('With standard deviation: %3f' % test_std)
    
    val_means_pquit.append(val_mean)
    val_stds_pquit.append(val_std)
    test_means_pquit.append(test_mean)
    test_stds_pquit.append(test_std)

print('\n') 
print(tabulate({'p_quit': np.linspace(0.1, 0.9, 9), 'RMSE': test_means_pquit, 'std': test_stds_pquit}, headers='keys'))


- This script take as input a kernel matrix
and returns the classification or regression performance
- The kernel matrix can be calculated using any of the graph kernels approaches
- The criteria used for prediction are SVM for classification and kernel Ridge regression for regression
- For predition we divide the data in training, validation and test. For each split, we first train on the train data, 
then evaluate the performance on the validation. We choose the optimal parameters for the validation set and finally
provide the corresponding performance on the test set. If more than one split is performed, the final results 
correspond to the average of the performances on the test sets. 

@references
    https://github.com/eghisu/GraphKernels/blob/master/GraphKernelsCollection/python_scripts/compute_perf_gk.py


 Loading dataset from file...
[ -23.7   14.    37.3  109.7   10.8   39.    42.    66.6  135.   148.5
   40.    34.6   32.    63.    53.5   67.    64.4   84.7   95.5   92.
  


 Mean performance on val set: 30.572040
With standard deviation: 11.057046

 Mean performance on test set: 24.715650
With standard deviation: 4.891587


    std     RMSE    p_quit
-------  -------  --------
7.749    18.5188       0.1
6.59104  17.8991       0.2
7.10161  18.3924       0.3
6.24807  19.6233       0.4
6.29951  19.9936       0.5
6.26173  20.5466       0.6
6.33531  21.7018       0.7
6.10246  23.1489       0.8
4.89159  24.7157       0.9
